<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Images" data-toc-modified-id="Images-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Images</a></span></li></ul></li><li><span><a href="#train-val-split" data-toc-modified-id="train-val-split-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>train val split</a></span><ul class="toc-item"><li><span><a href="#Texts" data-toc-modified-id="Texts-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Texts</a></span><ul class="toc-item"><li><span><a href="#No-train" data-toc-modified-id="No-train-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>No train</a></span></li></ul></li><li><span><a href="#Effnet-image" data-toc-modified-id="Effnet-image-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Effnet image</a></span><ul class="toc-item"><li><span><a href="#b0" data-toc-modified-id="b0-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>b0</a></span></li><li><span><a href="#TF-b3" data-toc-modified-id="TF-b3-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>TF b3</a></span></li><li><span><a href="#TF-b0" data-toc-modified-id="TF-b0-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>TF b0</a></span></li><li><span><a href="#b2" data-toc-modified-id="b2-1.2.4"><span class="toc-item-num">1.2.4&nbsp;&nbsp;</span>b2</a></span></li><li><span><a href="#b3" data-toc-modified-id="b3-1.2.5"><span class="toc-item-num">1.2.5&nbsp;&nbsp;</span>b3</a></span></li></ul></li></ul></li></ul></div>

## Images

In [1]:
from imports import *
from utils import load_data
from image_train.data import create_dl, ImageDS
from image_train.model import EMBRes
from image_train.arcface import ArcMarginProduct
from image_train.train import get_hparams, train
np.random.seed(1337)
device = torch.device('cuda')

In [2]:
df, train_df, val_df, train_labels, val_labels = load_data()

In [2]:
df = pd.read_csv('data/train.csv')
df['label_group'] = df['label_group'].astype('category').cat.codes
df['image_phash'] = df['image_phash'].astype('category').cat.codes
np.random.seed(1337)

# train val split
ph = np.random.permutation(df['image_phash'].unique())

train_perc = 0.3
train_idx = int(train_perc * len(ph))

train_labels = ph[:train_idx]
val_labels = ph[train_idx:]

train_df = df[df['image_phash'].isin(train_labels)]
val_df = df[df['image_phash'].isin(val_labels)]

In [3]:
# creating dataloaders
small_images_dir = 'data/small_train_images/'

tr_dl = create_dl(train_df, small_images_dir)
tr_test_dl = create_dl(train_df, small_images_dir, shuffle=False)
val_dl = create_dl(val_df, small_images_dir, shuffle=False)
full_dl = create_dl(val_df, small_images_dir, shuffle=False)

***Embeddings normalization is not done in the model but in the arcface metric***

In [4]:
vision_model = 'resnet50'
model = EMBRes(vision_model).to(device)
metric_fc = ArcMarginProduct(512, df['label_group'].nunique(), s=30, m=0.5, easy_margin=False).to(device)

In [5]:
n_epochs, lf, params, optimizer, sched, train_transforms, val_transforms = get_hparams(tr_dl, model, metric_fc, n_epochs=5, lr=1e-4)

In [6]:
train(model, optimizer, lf, sched, metric_fc, tr_dl, val_dl, n_epochs, val_df, 
      train_transforms, val_transforms, save_path='data/tests_model_image/test', val_first=False)

f1 score : 0.6415408431740804 | precision : 0.9948976052261963 | recall : 0.5302041878051954


f1 score : 0.6592646124322654 | precision : 0.9928683064355703 | recall : 0.5532492114553005


f1 score : 0.6734069451839727 | precision : 0.9896338779349546 | recall : 0.5720793159076134


f1 score : 0.6844808519268232 | precision : 0.9806071326428477 | recall : 0.590947941113439


f1 score : 0.6921901527146699 | precision : 0.9646927470743845 | recall : 0.6115212998768145


...

f1 score : 0.6279544857403415 | precision : 0.7768038032109599 | recall : 0.6865052358291729


f1 score : 0.5348809750843183 | precision : 0.6145604958039327 | recall : 0.7196928839609666


f1 score : 0.4030325291005304 | precision : 0.41855023208465375 | recall : 0.7539900652094109


f1 score : 0.2668994354599007 | precision : 0.23717663014191848 | recall : 0.7829678489983581


KeyboardInterrupt: 

## Texts

In [1]:
from utils import compute_f1, load_data
from text_train.imports import *
from text_train.data import TextDS, text_to_device
from text_train.arcface import ArcMarginProduct
from text_train.model import EMBBert
from text_train.train import train, get_hparams
np.random.seed(1337)
device = torch.device('cuda')

In [2]:
df, train_df, val_df, train_labels, val_labels = load_data()

In [3]:
bs = 64

In [4]:
# creating dataloaders

language_model = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(language_model)

tr_ds = TextDS(df, tokenizer)
tr_dl = DataLoader(tr_ds, batch_size = bs, shuffle = True, pin_memory = True)

val_ds = TextDS(val_df, tokenizer)
val_dl = DataLoader(val_ds, batch_size = bs, shuffle = False, pin_memory = True)

In [14]:
full_ds = TextDS(df, tokenizer)
full_dl = DataLoader(full_ds, batch_size = bs, shuffle = False, pin_memory = True)

In [5]:
# model
model = EMBBert().to(device)
metric_fc = ArcMarginProduct(512, df['label_group'].nunique(), s=30, m=0.5, easy_margin=False).to(device)

In [6]:
n_epochs, lf, params, optimizer, sched = get_hparams(tr_dl, model, metric_fc, lr=1e-4, n_epochs=10)

In [7]:
train(model, optimizer, lf, sched, metric_fc, tr_dl, val_dl, n_epochs, val_df, save_path='data/text_models/bert_arcface')

f1 score : 0.5393978641241446 | precision : 0.9857564428169381 | recall : 0.41386682242254036


f1 score : 0.5856510054626642 | precision : 0.920408225449109 | recall : 0.495706869016028


f1 score : 0.4729467528359827 | precision : 0.5668552154900484 | recall : 0.6158096204246976


f1 score : 0.2231655320077384 | precision : 0.1947180422016007 | recall : 0.7040833473677098


f1 score : 0.1388632290461438 | precision : 0.09458227186598418 | recall : 0.728945658078682


...

f1 score : 0.9525562376704738 | precision : 0.9491205861934432 | recall : 0.9759714671514266


f1 score : 0.9251674046018573 | precision : 0.8984145201598348 | recall : 0.9880296491816091


f1 score : 0.8412485536104918 | precision : 0.779403601600667 | recall : 0.9944305399575002


f1 score : 0.5832083876584692 | precision : 0.4662026253369048 | recall : 0.9972661008200207
Saved best model ep 9 with f score : 0.9535306686362947
Ep 9: Train loss 3.900527693442444 | Val f score 0.9535306686362947



([],
 ([[24.57255744934082,
    24.56796646118164,
    24.540767669677734,
    24.28423309326172,
    24.556447982788086,
    24.274240493774414,
    24.831405639648438,
    24.775949478149414,
    24.633424758911133,
    24.360225677490234,
    24.2344970703125,
    24.421627044677734,
    24.5714168548584,
    24.737096786499023,
    24.879196166992188,
    24.400041580200195,
    24.54337501525879,
    24.735254287719727,
    24.394906997680664,
    24.59442710876465,
    24.50320816040039,
    24.51884651184082,
    24.67342758178711,
    24.676912307739258,
    24.443143844604492,
    24.526514053344727,
    24.5673770904541,
    24.59941291809082,
    24.614107131958008,
    24.20348358154297,
    24.39006996154785,
    24.552316665649414,
    24.651517868041992,
    24.339414596557617,
    24.634601593017578,
    24.592662811279297,
    24.400575637817383,
    24.446582794189453,
    24.449268341064453,
    24.454723358154297,
    24.583044052124023,
    24.60056495666504,
    2

In [10]:
torch.save(model.text_embedor, 'data/text_models/bert_final_10ep_text_emb.pth')

In [11]:
torch.save(model.state_dict(), 'data/text_models/bert_final_10ep_sd.pth')

In [9]:
torch.save(tokenizer, 'data/text_models/bert_tokenizer.pth')

In [16]:
with torch.no_grad():
    pbar = tqdm(full_dl, leave=False)
    text_embeddings = []
    for txts, _ in pbar:
        txts = text_to_device(txts, device)
        feature =  model(txts)
        text_embeddings.append(feature)
    text_embeddings = F.normalize(torch.cat(text_embeddings, 0))

In [22]:
compute_f1(text_embeddings, df['label_group'], 0.7)

f1 score : 0.8866721749430263 | precision : 0.9687548716957335 | recall : 0.8586787032327977


(0.8866721749430263, 0.9687548716957335, 0.8586787032327977)

### No train

In [8]:
full_ds = SoftMaxDS(df, tokenizer)
full_dl = DataLoader(full_ds, batch_size = bs, shuffle = False, pin_memory = True)

In [15]:
model = BertModel.from_pretrained('bert-base-uncased').to(device)

In [34]:
with torch.no_grad():
    pbar = tqdm(full_dl, leave=False)
    embs = []
    for txts, _ in pbar:
        txts = text_to_device(txts, device)
        feature =  model(txts['input_ids'], 
                                     attention_mask=txts['attention_mask'])[0][:,0.,:]
        embs.append(feature)
    embs = torch.cat(embs, 0)

In [17]:
torch.save(embs, 'data/text_embs/bert_notrain_embs.pth')

In [35]:
embs.shape

torch.Size([34250, 40])

In [36]:
compute_f1(embs, df['label_group'], 2.9)

f1 score : 0.044660520120419656 | precision : 0.02503708029195983 | recall : 0.37171434782554214


(0.044660520120419656, 0.02503708029195983, 0.37171434782554214)

## Effnet image

In [1]:
import os
import random
import functools
from functools import partial
import PIL

import numpy as np 
import pandas as pd

from tqdm.notebook import tqdm
import math

import torch
from torch.nn.parameter import Parameter
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

from transformers import BertTokenizer
from transformers import BertModel

import matplotlib.pyplot as plt

import timm
np.random.seed(1337)


In [2]:
# load in data

df = pd.read_csv('data/train.csv')
small_images_dir = 'data/small_train_images/'
n_classes = df['label_group'].nunique()

labels = np.random.permutation(df['label_group'].unique())

train_perc = 0.7
train_idx = int(train_perc * len(labels))

train_labels = labels[:train_idx]
val_labels = labels[train_idx:]

In [3]:
train_df = df[df['label_group'].isin(train_labels)]
val_df = df[df['label_group'].isin(val_labels)]

In [4]:
val_df.head()

,posting_id,image,image_phash,title,label_group
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188
6,train_1802986387,00144a49c56599d45354a1c28104c039.jpg,f815c9bb833ab4c8,Jubah anak size 1-12 thn,1835033137
8,train_86570404,0019a3c6755a194cb2e2c12bfc63972e.jpg,ea9af4f483249972,"[LOGU] Tempelan kulkas magnet angka, tempelan ...",2359912463


In [5]:
class SoftMaxDS(Dataset):
    def __init__(self, data, images_path, return_triplet = True):
        super().__init__()
        self.imgs = data['image'].tolist()
        self.unique_labels = data['label_group'].unique().tolist()
        self.labels = data['label_group'].astype('category')
        self.label_codes = self.labels.cat.codes
        
        self.images_path = images_path
        
    def __getitem__(self, idx):
        
        img = self._get_item(idx)
        label = self.label_codes.iloc[idx]
        return img, label
    def __len__(self):
        return len(self.imgs)
    
    def _get_item(self, idx):
        im = PIL.Image.open(os.path.join(self.images_path, self.imgs[idx]))
        im = torch.tensor(np.array(im) / 255.0, dtype = torch.float).permute(2,0,1)
        return im

In [6]:
# creating dataloaders

vision_model = 'resnet50'

bs = 64
tr_ds = SoftMaxDS(train_df, small_images_dir)
tr_dl = DataLoader(tr_ds, batch_size = bs, shuffle = True, pin_memory = True)
tr_test_dl = DataLoader(tr_ds, batch_size = bs, shuffle = False, pin_memory = True)

val_ds = SoftMaxDS(val_df, small_images_dir)
val_dl = DataLoader(val_ds, batch_size = bs, shuffle = False, pin_memory = True)

full_ds = SoftMaxDS(df, small_images_dir)
full_dl = DataLoader(full_ds, batch_size = bs, shuffle = False, pin_memory = True)

device = torch.device('cuda')

val_label_count = val_df['label_group'].value_counts()

In [7]:
class EMBRes(nn.Module) :
    def __init__(self, pretrained_image_embedor='resnet50',
                output_dim=512) :
        super(EMBRes, self).__init__()
        self.image_embedor = timm.create_model(pretrained_image_embedor, pretrained=True, n_classes=0)
    
    def _get_embs(self, x) :
        images = x
        out_images = self.image_embedor.forward_features(images)
        out_images = self.image_pool(out_images).squeeze()
        #return F.normalize(out_images, dim=-1)
        return out_images
    
    def forward(self, x) :
        out_images = self._get_embs(x)
        
        return self.head(out_images)

In [8]:
# https://github.com/ronghuaiyang/arcface-pytorch

class ArcMarginProduct(nn.Module):
    r"""Implement of large margin arc distance: :
        Args:
            in_features: size of each input sample
            out_features: size of each output sample
            s: norm of input feature
            m: margin
            cos(theta + m)
        """
    def __init__(self, in_features, out_features, s=30.0, m=0.50, easy_margin=False):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.weight = Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt((1.0 - torch.pow(cosine, 2)).clamp(0, 1))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size(), device='cuda')
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)  # you can use torch.where if your torch.__version__ is 0.4
        output *= self.s
        # print(output)

        return output


In [9]:
def compute_f1(embeddings, ls, threshold) :
    dists = torch.cdist(embeddings, embeddings)
    distances, indices = torch.topk(dists, 50, dim=1, largest=False)
    
    THRESHOLD = threshold
    preds = [[] for _ in range(embeddings.shape[0])]
    for i in tqdm(range(distances.shape[0]), leave=False) :
        IDX = torch.where(distances[i,]<THRESHOLD)[0]
        IDS = indices[i,IDX]
        preds[i] = IDS.cpu().numpy()
            
    label_counts = ls.value_counts()
    f_score = 0 
    precision = 0
    recall = 0
    for i in tqdm(range(embeddings.shape[0]), leave=False) :
        tp = 0
        fp = 0
        true_label = ls.iloc[i]
        for index in preds[i] :
            if ls.iloc[index] == true_label :
                tp += 1
            else :
                fp += 1
        fn = label_counts[true_label] - tp
        #print(label_counts[true_label]-1, tp)
        f_score += 2*tp / (label_counts[true_label] + len(preds[i]))
        precision += tp / len(preds[i])
        recall += tp/ label_counts[true_label]
    f_score = f_score/embeddings.shape[0]
    precision = precision/embeddings.shape[0]
    recall = recall/embeddings.shape[0]
    
    print('f1 score : {} | precision : {} | recall : {}'.format(f_score, precision, recall))
    return f_score, precision, recall

***Embeddings normalization is not done in the model but in the arcface metric***

In [10]:
#model = EMBRes('efficientnet_b0').to(device)
#metric_fc = ArcMarginProduct(512, df['label_group'].nunique(), s=30, m=0.5, easy_margin=False).to(device)

In [11]:

normalize = transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                 std=(0.229, 0.224, 0.225))

train_transforms = transforms.Compose([transforms.ColorJitter(.3,.3,.3),
                                       transforms.RandomRotation(5),
                                       transforms.RandomCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       normalize
                                       ])

val_transforms = transforms.Compose([normalize
                                     ])

n_epochs = 30

lf = nn.CrossEntropyLoss()

lr = 1e-2
wd = 0
no_decay = ["bias", "BatchNorm2d.weight", "BatchNorm2d.bias", "LayerNorm.weight", 'LayerNorm.bias',
            "BatchNorm1d.weight", "BatchNorm1d.bias"]

params = list(model.named_parameters()) + list(metric_fc.named_parameters())
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in params if not any(nd in n for nd in no_decay)],
        "weight_decay": wd,
    },
    {
        "params": [p for n, p in  model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=lr)

# learning rate scheduler
sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr =lr, pct_start = 0.3, #anneal_strategy = 'linear',
                                            total_steps = int(n_epochs * len(tr_dl)))

NameError: name 'model' is not defined

### b0

In [24]:
small_images_dir = 'data/small_train_images_600/'
full_ds = SoftMaxDS(df, small_images_dir)
full_dl = DataLoader(full_ds, batch_size = bs, shuffle = False, pin_memory = True)

In [25]:
model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=0).to(device)

In [26]:
with torch.no_grad():
    pbar = tqdm(full_dl, leave=False)
    embs = []
    for imgs, _ in pbar:
        imgs = val_transforms(imgs.to(device))
        feature = model(imgs)
        embs.append(feature)
    embs = torch.cat(embs, 0)

In [38]:
torch.save(embs, 'data/effnet_embs/embs_b0_notrain_600.pth')

In [37]:
compute_f1(embs, df['label_group'], 6.6)

f1 score : 0.6363438917214435 | precision : 0.9273578644382607 | recall : 0.558110326058612


(0.6363438917214435, 0.9273578644382607, 0.558110326058612)

### TF b3

In [12]:
embs = torch.load('data/effnet_embs/image_embeddings.pth')

In [14]:
embs = torch.from_numpy(embs).float()

In [19]:
compute_f1(embs, df['label_group'], 5)

f1 score : 0.6409645357216606 | precision : 0.9576403504043125 | recall : 0.5480520553212448


(0.6409645357216606, 0.9576403504043125, 0.5480520553212448)

### TF b0

In [10]:
embs = torch.load('data/effnet_embs/tf_embs_b0_256.pth')

In [13]:
embs = torch.from_numpy(embs).float().to(device)

TypeError: expected np.ndarray (got Tensor)

In [21]:
compute_f1(embs, df['label_group'], 7.5)

f1 score : 0.642667609875153 | precision : 0.9084886567491639 | recall : 0.5867818006410631


(0.642667609875153, 0.9084886567491639, 0.5867818006410631)

### b2

In [10]:
model = timm.create_model('efficientnet_b2', pretrained=True, num_classes=0).to(device)

In [13]:
with torch.no_grad():
    pbar = tqdm(full_dl, leave=False)
    embs = []
    for imgs, _ in pbar:
        imgs = val_transforms(imgs.to(device))
        feature = model(imgs)
        embs.append(feature)
    embs = torch.cat(embs, 0)

In [22]:
compute_f1(embs, df['label_group'], 10)

f1 score : 0.6196357491479187 | precision : 0.9479240435254366 | recall : 0.5289644536998594


(0.6196357491479187, 0.9479240435254366, 0.5289644536998594)

In [24]:
#torch.save(embs, 'data/effnet_embs/embs_b2_notrain.pth')

### b3

In [ ]:
small_images_dir = 'data/small_train_images_300/'
full_ds = SoftMaxDS(df, small_images_dir)
full_dl = DataLoader(full_ds, batch_size = bs, shuffle = False, pin_memory = True)

In [12]:
model = timm.create_model('efficientnet_b3', pretrained=True, num_classes=0).to(device)

In [13]:
with torch.no_grad():
    pbar = tqdm(full_dl, leave=False)
    embs = []
    for imgs, _ in pbar:
        imgs = val_transforms(imgs.to(device))
        feature = model(imgs)
        embs.append(feature)
    embs = torch.cat(embs, 0)

In [22]:
compute_f1(embs, df['label_group'], 8.1)

f1 score : 0.60674607145152 | precision : 0.9222303967400572 | recall : 0.5332024273267191


(0.60674607145152, 0.9222303967400572, 0.5332024273267191)

In [15]:
torch.save(embs, 'data/effnet_embs/embs_b3_notrain.pth')

In [11]:
embs = torch.load('data/effnet_embs/embs_b3_notrain.pth')